In [1]:
## 여러 입력 값을 갖는 로지스틱 회귀 실제 값 적용하기
#7시간 공부하고 과외를 6번 받은 학생의 합격 가능성
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

#실행할 때마다 같은 결과를 출력하기 위한 seed값 설정
seed = 0
np.random.seed(seed)    #넘파이, 텐서플로우 seed값 초기화
tf.set_random_seed(seed)

#x, y의 데이터 값
x_data = np.array([[2,3],[4,3],[6,4],[8,6],[10,7],[12,8],[14,19]])    #[공부시간, 과외횟수]
y_data = np.array([0,0,0,1,1,1,1]).reshape(7,1)    #가로 7, 세로1로 만든다. 합격/불합격 결과

#입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float64, shape=[None, 2])    #앞은 상관없고, 1차원 요소는 2개
Y = tf.placeholder(tf.float64, shape=[None, 1])    #앞은 상관없고, 1차원 요소는 1개

#기울기 a와 바이어스 b의 값을 임의로 정함
a = tf.Variable(tf.random_uniform([2,1], dtype=tf.float64))
#기울기 2개니까 a1, a2 묶어서 [2,1](하나로 표현된 식이 2개 있다)
#a는 [2,1] 형태를 가짐 a1, a2
#[2,1] 의미 : 들어오는 값은 2개, 나가는 값은 1개
b = tf.Variable(tf.random_uniform([1], dtype=tf.float64))

#y 시그모이드 함수의 방정식을 세움
y = tf.sigmoid(tf.matmul(X, a) + b)    #예측치. () 안 부분은 -z식

#오차를 구하는 함수
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))

#학습률 값
learning_rate = 0.1

#오차를 최소로 하는 값 찾기
gradient_decent = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

predicted = tf.cast(y > 0.5, dtype=tf.float64) 
#y가 0.5보다 크면 1, 아니면 0(텐서플로우의 cast연산자에 의해)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))
#텐서플로우의 평균(predicted와 Y가 같으면 1, 아니면 0인 값들의 평균)

#학습
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3001):
        a_, b_, loss_, _ = sess.run([a, b, loss, gradient_decent], feed_dict={X:x_data, Y:y_data})
        #플레이스홀더가 있으면 feed_dict로 값을 넣어준다
        if (i+1) % 300 == 0:
            print("step = %d, a1 = %.4f, a2 = %.4f, b = %.4f, loss = %.4f"
            %(i+1, a_[0], a_[1], b_, loss_))
    
    
    print()
    #추가 코드(sessoin안에 있어야 함)
    print("predicted = ", sess.run(predicted, feed_dict={X:x_data})) #
    #다른 값 테스트
    p_val, h_val = sess.run([predicted, y], feed_dict={X:[[1, 5], [10, 5], [4, 5]]})
    print("check predicted = ", p_val)    #연산된 값을 bool로 변환한 값
    print("check hypothesis = ", h_val)   #계산값
    #정확도 측정
    h, c, a = sess.run([y, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis : ", h, "\nCorrect (Y) :", c, "\nAccuracy : ",a)    #예측치, 결과, 정확도 인듯....
    
    #===========================================여기부터 오늘 코드=============================================
    new_x = np.array([7,6]).reshape(1,2)    #7,6은 각각 공부한 시간과 과외수업횟수
    new_y, new_y_result = sess.run([y, predicted], feed_dict={X:new_x})
    #새 합격 가능성, 그걸 predicted한게 y_result, 데이터를 플레이스 홀더에 넣어준다
    
    print("공부한 시간 : %d, 과외 수업 횟수 : %d" %(new_x[:,0], new_x[:,1]))
    print("합격 가능성 : %6.2f %%" %(new_y*100))    #%하나만 쓰면 %글자 출력 안되니까 %%쓴거다
    print("예측 결과 :",new_y_result[0])

Instructions for updating:
non-resource variables are not supported in the long term
step = 300, a1 = 0.4065, a2 = 0.0859, b = -2.8088, loss = 0.2459
step = 600, a1 = 0.5048, a2 = 0.2208, b = -4.2800, loss = 0.1711
step = 900, a1 = 0.4974, a2 = 0.4251, b = -5.2946, loss = 0.1351
step = 1200, a1 = 0.4489, a2 = 0.6468, b = -6.0918, loss = 0.1121
step = 1500, a1 = 0.3886, a2 = 0.8598, b = -6.7544, loss = 0.0958
step = 1800, a1 = 0.3273, a2 = 1.0563, b = -7.3229, loss = 0.0836
step = 2100, a1 = 0.2689, a2 = 1.2353, b = -7.8217, loss = 0.0741
step = 2400, a1 = 0.2148, a2 = 1.3980, b = -8.2662, loss = 0.0665
step = 2700, a1 = 0.1651, a2 = 1.5461, b = -8.6673, loss = 0.0603
step = 3000, a1 = 0.1197, a2 = 1.6814, b = -9.0329, loss = 0.0552

predicted =  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
check predicted =  [[0.]
 [1.]
 [0.]]
check hypothesis =  [[0.37624807]
 [0.63890395]
 [0.46336593]]

Hypothesis :  [[0.02298783]
 [0.02901783]
 [0.16945959]
 [0.88218362]
 [0.98081412]
 [0.99714306]
 

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

#배열 슬라이싱 응용
xy = np.loadtxt("/content/data-03-diabetes.csv", delimiter=",")
x_data = xy[:, 0:-1]    #[전체줄, 각 줄 데이터 중 0~맨 끝에거 앞에까지]
y_data = xy[:, [-1]]    #[전체줄, 각 줄 데이터 중 맨 끝에거]

#Placeholders : Shape주의! 총 8개의 x_data와 1개의 y_data
X = tf.placeholder(tf.float32, shape=[None,8])#
Y = tf.placeholder(tf.float32, shape=[None,1])#

W = tf.Variable(tf.random_normal([8,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')
#random_normal은 어느정도 예측할 수 있을때 그 값을 중심으로 하지만
#대부분은 예측할 수 없으니까 uniform을 쓰는게 좋다

#Hypothesis예측값
hyp = tf.sigmoid(tf.matmul(X, W) + b)

#Cost/Loss function
cost = -tf.reduce_mean(Y * tf.log(hyp) + (1 - Y) * tf.log(1-hyp))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#정확도 hyp > 0.5 else False
predicted = tf.cast(hyp > 0.5, dtype=tf.float32)#
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))#

#세션 시작
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, cost_val)
            # 10000 0.480384
        
    #정확도 77%
    _, _, a = sess.run([hyp, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    #hyp, predicted은 연산을 위해 필요한거니까 출력 안해도 됨
    #a = sess.run(accuracy, feed_dict={X:x_data, Y:y_data})로 해도 문제는 안됨
    #graph가 구성되어 있으므로
    print("Accuracy :", a)
    #Accuracy : 0.769433

0 1.0982647
200 0.7647606
400 0.6994914
600 0.67361456
800 0.655156
1000 0.6394628
1200 0.6256324
1400 0.61334455
1600 0.60239047
1800 0.5925983
2000 0.58382064
2200 0.57592976
2400 0.56881547
2600 0.562383
2800 0.55655044
3000 0.5512471
3200 0.546412
3400 0.54199237
3600 0.5379424
3800 0.5342225
4000 0.53079784
4200 0.52763826
4400 0.52471733
4600 0.5220114
4800 0.5195003
5000 0.51716566
5200 0.5149916
5400 0.5129636
5600 0.5110692
5800 0.5092968
6000 0.50763637
6200 0.5060787
6400 0.5046156
6600 0.50323975
6800 0.5019443
7000 0.50072324
7200 0.49957117
7400 0.498483
7600 0.49745417
7800 0.49648055
8000 0.49555835
8200 0.49468398
8400 0.49385443
8600 0.49306655
8800 0.49231783
9000 0.49160567
9200 0.49092776
9400 0.49028206
9600 0.48966655
9800 0.4890795
10000 0.48851916
Accuracy : 0.76811594


In [5]:
# 퍼셉트론
#AND, OR, NAND를 구현하는 로직연산
#퍼셉트론은 XOR못한다

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
#y_data = np.array([[0],[0],[0],[1]], dtype=np.float32)    #And를 구현하는 로직
#y_data = np.array([[0],[1],[1],[1]], dtype=np.float32)    #OR를 구현하는 로직
y_data = np.array([[1],[1],[1],[0]], dtype=np.float32)     #NAND를 구현하는 로직

X = tf.placeholder(tf.float32, shape=[None,2])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([2,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hyp = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(Y * tf.log(hyp) + (1 - Y) * tf.log(1-hyp))   #오차
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hyp > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print("step =", step, "cost =", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            print('W =',sess.run(W), "b =",sess.run(b))

    h, p, a = sess.run([hyp, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis :",h,"\nCorrect",p,"\nAccuracy :", a)
    
    #실제값 확인
    print("\n다른값 테스트")
    new_x = np.array([0,1]).reshape(1,2)
    new_y = sess.run(hyp, feed_dict={X:new_x})
    h, c = sess.run([hyp, predicted], feed_dict={X:new_x, Y:new_y})
    print('Hyothesis :', h, "\nCorrect(Y) :",c)

step = 0 cost = 0.8677834
W = [[1.5349876 ]
 [0.44916767]] b = [0.31414393]
step = 1000 cost = 0.5466203
W = [[ 0.17718239]
 [-0.47129968]] b = [0.8692014]
step = 2000 cost = 0.40897816
W = [[-0.5855367]
 [-0.9635568]] b = [1.6038785]
step = 3000 cost = 0.33058867
W = [[-1.116324 ]
 [-1.3425575]] b = [2.1993902]
step = 4000 cost = 0.2795989
W = [[-1.5207171]
 [-1.6602919]] b = [2.6932638]
step = 5000 cost = 0.24325544
W = [[-1.8479936]
 [-1.9366994]] b = [3.116632]
step = 6000 cost = 0.21572495
W = [[-2.1243367]
 [-2.1822994]] b = [3.4884691]
step = 7000 cost = 0.19398177
W = [[-2.3647373]
 [-2.4035912]] b = [3.8208263]
step = 8000 cost = 0.1762886
W = [[-2.578371]
 [-2.605028]] b = [4.1217985]
step = 9000 cost = 0.16156566
W = [[-2.7711957]
 [-2.789877 ]] b = [4.397102]
step = 10000 cost = 0.1490995
W = [[-2.9472878]
 [-2.9606347]] b = [4.65094]

Hypothesis : [[0.99053776]
 [0.8442643 ]
 [0.8460111 ]
 [0.22149372]] 
Correct [[1.]
 [1.]
 [1.]
 [0.]] 
Accuracy : 1.0

다른값 테스트
Hyothesis :